In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_10%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("max_count:",max_count)
#     print("max_vals:",max_vals)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,5)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.0,50.0,0.0,98.0
1,0.0,13.0,3250.0,28.0
2,1.0,16.0,4000.0,0.0
3,2.0,20.0,5000.0,45.0
4,1.0,24.0,6000.0,77.0
...,...,...,...,...
743,23.0,2.0,500.0,0.0
744,0.0,2.0,0.0,52.0
745,0.0,0.0,0.0,62.0
746,39.0,1.0,250.0,0.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 379


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)

Kolom : 1
[2.0, 4.0, 1.0, 2.0, 14.0]
mean :  4.6
standar deviasi :  4.8
fuzzy :  [(2.0, 0.8635649403472458), (4.0, 0.9922187419902069), (1.0, 0.7548616143394908), (2.0, 0.8635649403472458), (14.0, 0.14699805787397555)]
fuzzy :  None
pembilang :  10.235969153924959
penyebut :  3.6212082948981643
rata-rata tertimbang :  2.826672292876987
 
[4.0, 0.0, 4.0, 9.0, 11.0]
mean :  5.6
standar deviasi :  3.9293765408777
fuzzy :  [(4.0, 0.920449667968578), (0.0, 0.3622434622703331), (4.0, 0.920449667968578), (9.0, 0.6877612530712153), (11.0, 0.3889896240845836)]
fuzzy :  None
pembilang :  17.83233448631998
penyebut :  3.279893675363288
rata-rata tertimbang :  5.436863585019973
 
[11.0, 1.0, 5.0, 6.0, 11.0]
mean :  6.8
standar deviasi :  3.815756805667783
fuzzy :  [(11.0, 0.5456888038579911), (1.0, 0.31502603206372876), (5.0, 0.8947130738672752), (6.0, 0.978263964313968), (11.0, 0.5456888038579911)]
fuzzy :  None
pembilang :  22.663328872159717
penyebut :  3.279380677960954
rata-rata tertimbang : 

[2.0, 11.0, 4.0, 4.0, 4.0]
mean :  5.0
standar deviasi :  3.0983866769659336
fuzzy :  [(2.0, 0.6258144246994224), (11.0, 0.15338478317675563), (4.0, 0.9492548855738783), (4.0, 0.9492548855738783), (4.0, 0.9492548855738783)]
fuzzy :  None
pembilang :  14.329920091229695
penyebut :  3.626963864597813
rata-rata tertimbang :  3.950940959490015
 
[2.0, 2.0, 2.0, 2.0, 4.0]
mean :  2.4
standar deviasi :  0.8
fuzzy :  [(2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (2.0, 0.8825083402897206), (4.0, 0.13536335042701722)]
fuzzy :  None
pembilang :  7.601520124025834
penyebut :  3.6653967115858994
rata-rata tertimbang :  2.0738601363389395
 
[2.0, 0.0, 11.0, 16.0, 14.0]
mean :  8.6
standar deviasi :  6.437390775772433
fuzzy :  [(2.0, 0.5912450825279498), (0.0, 0.4097197079683316), (11.0, 0.9328686563681979), (16.0, 0.5165176699672577), (14.0, 0.7034225526018996)]
fuzzy :  None
pembilang :  29.556243841008794
penyebut :  3.1537736694336367
rata-rata tertimbang :  9

[0.0, 3.0, 0.0, 0.0, 23.0]
mean :  5.2
standar deviasi :  8.97552226892675
fuzzy :  [(0.0, 0.8455163081980631), (3.0, 0.9704100088082527), (0.0, 0.8455163081980631), (0.0, 0.8455163081980631), (23.0, 0.13997650187324653)]
fuzzy :  None
pembilang :  6.130689569509428
penyebut :  3.6469354352756884
rata-rata tertimbang :  1.6810524009306957
 
[0.0, 0.0, 6.0, 14.0, 21.0]
mean :  8.2
standar deviasi :  8.207313811473277
fuzzy :  [(0.0, 0.6071025759113177), (0.0, 0.6071025759113177), (6.0, 0.964714920174215), (14.0, 0.7790522224111285), (21.0, 0.29640576870954116)]
fuzzy :  None
pembilang :  22.919541777701454
penyebut :  3.25437806311752
rata-rata tertimbang :  7.042679532981413
 
[0.0, 1.0, 0.0, 2.0, 4.0]
mean :  1.4
standar deviasi :  1.4966629547095764
fuzzy :  [(0.0, 0.6456778149157842), (1.0, 0.9649195174720684), (0.0, 0.6456778149157842), (2.0, 0.9227944093228095), (4.0, 0.22118139566465603)]
fuzzy :  None
pembilang :  3.6952339187763115
penyebut :  3.4002509522911026
rata-rata terti

[0.0, 16.0, 9.0, 14.0, 14.0]
mean :  10.6
standar deviasi :  5.782732917920384
fuzzy :  [(0.0, 0.18640198430222035), (16.0, 0.6466438641419613), (9.0, 0.9624496333409749), (14.0, 0.8412814882191207), (14.0, 0.8412814882191207)]
fuzzy :  None
pembilang :  42.564230196475535
penyebut :  3.4780584582233978
rata-rata tertimbang :  12.237928346442304
 
[0.0, 21.0, 21.0, 4.0, 2.0]
mean :  9.6
standar deviasi :  9.393614852653903
fuzzy :  [(0.0, 0.5932389538868104), (21.0, 0.4788708356440437), (21.0, 0.4788708356440437), (4.0, 0.8372113218089523), (2.0, 0.720899690786467)]
fuzzy :  None
pembilang :  24.90321976585858
penyebut :  3.109091637770317
rata-rata tertimbang :  8.009805649767824
 
[0.0, 11.0, 1.0, 5.0, 6.910856377385035]
mean :  4.7821712754770065
standar deviasi :  4.01033976936759
fuzzy :  [(0.0, 0.4911984004941704), (11.0, 0.3006459403940973), (1.0, 0.6410310651518126), (5.0, 0.9985260841880697), (6.910856377385035, 0.8686115308654507)]
fuzzy :  None
pembilang :  14.94361636797891

[0.0, 14.0, 21.0, 0.0, 4.0]
mean :  7.8
standar deviasi :  8.352245207128439
fuzzy :  [(0.0, 0.6466039527864748), (14.0, 0.7592019308924243), (21.0, 0.2868706893318738), (0.0, 0.6466039527864748), (4.0, 0.9016877668439712)]
fuzzy :  None
pembilang :  20.259862575839175
penyebut :  3.240968292641219
rata-rata tertimbang :  6.251175805033394
 
[0.0, 2.0, 4.0, 4.0, 5.0]
mean :  3.0
standar deviasi :  1.7888543819998317
fuzzy :  [(0.0, 0.24509627314906654), (2.0, 0.855359184584789), (4.0, 0.855359184584789), (4.0, 0.855359184584789), (5.0, 0.5352961160122968)]
fuzzy :  None
pembilang :  11.230072425909373
penyebut :  3.3464699429157303
rata-rata tertimbang :  3.3557965908771243
 
[0.0, 4.0, 4.0, 23.0, 21.0]
mean :  10.4
standar deviasi :  9.604165762834375
fuzzy :  [(0.0, 0.5564178001609942), (4.0, 0.8009101845708342), (4.0, 0.8009101845708342), (23.0, 0.4229539952697087), (21.0, 0.5438944031577922)]
fuzzy :  None
pembilang :  27.55700583408361
penyebut :  3.1250865677301634
rata-rata tert

...

[0.0, 0.0, 64.0, 81.0, 74.0]
mean :  43.8
standar deviasi :  36.16849457746341
fuzzy :  [(0.0, 0.48037755989791003), (0.0, 0.48037755989791003), (64.0, 0.8556076633586067), (81.0, 0.5892697126519908), (74.0, 0.7057033189096826)]
fuzzy :  None
pembilang :  154.7117827790786
penyebut :  3.1113358147161003
rata-rata tertimbang :  49.725195861956664
 
[48.0, 0.0, 0.0, 25.0, 28.0]
mean :  20.2
standar deviasi :  18.29098138427788
fuzzy :  [(48.0, 0.3150922554861276), (0.0, 0.5434852839394497), (0.0, 0.5434852839394497), (25.0, 0.9661562432833172), (28.0, 0.9130944377267913)]
fuzzy :  None
pembilang :  64.84497860176722
penyebut :  3.281313504375136
rata-rata tertimbang :  19.76189672681572
 
[11.0, 16.0, 16.0, 16.0, 26.0]
mean :  17.0
standar deviasi :  4.898979485566356
fuzzy :  [(11.0, 0.4724032869055456), (16.0, 0.9793842968861823), (16.0, 0.9793842968861823), (16.0, 0.9793842968861823), (26.0, 0.18501376841789716)]
fuzzy :  None
pembilang :  57.01724038536308
penyebut :  3.5955699459819

[0.0, 0.0, 0.0, 2.0, 2.0]
mean :  0.8
standar deviasi :  0.9797958971132713
fuzzy :  [(0.0, 0.7165560753439463), (0.0, 0.7165560753439463), (0.0, 0.7165560753439463), (2.0, 0.4724032869055458), (2.0, 0.4724032869055458)]
fuzzy :  None
pembilang :  1.8896131476221831
penyebut :  3.0944747998429305
rata-rata tertimbang :  0.6106409875168789
 
[2.0, 0.0, 0.0, 0.0, 0.6106409875168789]
mean :  0.5221281975033758
standar deviasi :  0.7758599284013794
fuzzy :  [(2.0, 0.1630045235727641), (0.0, 0.797384029109614), (0.0, 0.797384029109614), (0.0, 0.797384029109614), (0.6106409875168789, 0.9935142797510487)]
fuzzy :  None
pembilang :  0.9326895880448292
penyebut :  3.5486708906526547
rata-rata tertimbang :  0.26282786338444963
 
[0.0, 0.0, 0.6106409875168789, 2.0, 2.0]
mean :  0.9221281975033758
standar deviasi :  0.9078854941545729
fuzzy :  [(0.0, 0.5970482508494107), (0.0, 0.5970482508494107), (0.6106409875168789, 0.942848596089894), (2.0, 0.49426262891160966), (2.0, 0.49426262891160966)]
fuzz

[0.0, 49.725195861956664, 0.0, 64.0, 81.0]
mean :  38.94503917239133
standar deviasi :  33.304698535835676
fuzzy :  [(0.0, 0.5047835822263655), (49.725195861956664, 0.9489683493462852), (0.0, 0.5047835822263655), (64.0, 0.7535615315579571), (81.0, 0.4506044522412321)]
fuzzy :  None
pembilang :  131.9145356892908
penyebut :  3.1627014975982055
rata-rata tertimbang :  41.70944864365743
 
[0.0, 0.0, 22.0, 23.0, 26.0]
mean :  14.2
standar deviasi :  11.668761716651858
fuzzy :  [(0.0, 0.476934728176021), (0.0, 0.476934728176021), (22.0, 0.7998024922019313), (23.0, 0.752509360664093), (26.0, 0.5997411050677757)]
fuzzy :  None
pembilang :  50.49663885547879
penyebut :  3.105922414285842
rata-rata tertimbang :  16.25817780354623
 
[0.0, 6.483922895160306, 14.0, 16.0, 14.0]
mean :  10.096784579032061
standar deviasi :  6.005263723326159
fuzzy :  [(0.0, 0.2433447778516433), (6.483922895160306, 0.8344753824457505), (14.0, 0.8096074304372627), (16.0, 0.616867426708143), (14.0, 0.8096074304372627)]

[0.0, 1.2212819750337578, 1.633541338095247, 2.0865718187986335, 4.0]
mean :  1.7882790263855277
standar deviasi :  1.3056844595660277
fuzzy :  [(0.0, 0.391480689669414), (1.2212819750337578, 0.9100297897507309), (1.633541338095247, 0.9930029195715646), (2.0865718187986335, 0.9742438948771495), (4.0, 0.23822766652155863)]
fuzzy :  None
pembilang :  5.719254818709262
penyebut :  3.5069849603904175
rata-rata tertimbang :  1.6308181766689303
 
[0.0, 14.0, 22.214140880749106, 23.0, 30.0]
mean :  17.842828176149823
standar deviasi :  10.262898893117274
fuzzy :  [(0.0, 0.22065248209794666), (14.0, 0.9323052821627347), (22.214140880749106, 0.9132914026868797), (23.0, 0.8814004436965541), (30.0, 0.49582311884374153)]
fuzzy :  None
pembilang :  68.48716160507458
penyebut :  3.4434727294878567
rata-rata tertimbang :  19.88898039429532
 
[0.0, 1.0442146659219271, 1.2212819750337578, 1.633541338095247, 4.0]
mean :  1.5798075958101863
standar deviasi :  1.3244931887659521
fuzzy :  [(0.0, 0.49102067

[0.0, 0.0, 23.0, 23.0, 23.0]
mean :  13.8
standar deviasi :  11.26765281680262
fuzzy :  [(0.0, 0.47240328690554567), (0.0, 0.47240328690554567), (23.0, 0.7165560753439464), (23.0, 0.7165560753439464), (23.0, 0.7165560753439464)]
fuzzy :  None
pembilang :  49.44236919873231
penyebut :  3.0944747998429305
rata-rata tertimbang :  15.977628643555898
 
[0.0, 15.977628643555898, 23.0, 23.0, 23.0]
mean :  16.99552572871118
standar deviasi :  8.922389176615864
fuzzy :  [(0.0, 0.1630045235727641), (15.977628643555898, 0.9935142797510488), (23.0, 0.7973840291096141), (23.0, 0.7973840291096141), (23.0, 0.7973840291096141)]
fuzzy :  None
pembilang :  70.89350022249555
penyebut :  3.548670890652655
rata-rata tertimbang :  19.977479571078835
 
[0.0, 21.0, 28.0, 14.0, 11.0]
mean :  14.8
standar deviasi :  9.453041838477178
fuzzy :  [(0.0, 0.293617546326059), (21.0, 0.8064908326588529), (28.0, 0.3772558137210872), (14.0, 0.9964257535848934), (11.0, 0.9223888485288249)]
fuzzy :  None
pembilang :  51.59

In [15]:
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.000000,50.000000,9458.714175,98.000000
1,2.826672,13.000000,3250.000000,28.000000
2,1.000000,16.000000,4000.000000,49.725196
3,2.000000,20.000000,5000.000000,45.000000
4,1.000000,24.000000,6000.000000,77.000000
...,...,...,...,...
743,23.000000,2.000000,500.000000,24.176427
744,8.961635,2.000000,578.559393,52.000000
745,4.598210,6.673322,1599.605286,62.000000
746,39.000000,1.000000,250.000000,23.140940


In [16]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
count,748.000000,748.000000,748.000000,748.000000
mean,9.023612,5.485855,1343.501207,33.188912
std,7.772916,5.840338,1426.633748,24.774523
min,1.000000,0.305320,9.232517,0.262828
25%,2.540195,2.000000,500.000000,14.000000
50%,6.227241,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,49.000000
max,74.000000,50.000000,11500.000000,98.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [38]:
x = dummy.values
y = df.iloc[:, -1].values

In [39]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=48)

In [40]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [41]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[119  24]
 [ 30  14]]
0.7112299465240641
